In [82]:
# autoreload
%load_ext autoreload
%autoreload 2

# fix path
import sys
sys.path.append("/home/jxm/reverse-training")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
from utils.analysis import load_results_as_df
import os


HOME_DIR = "/home/jxm/reverse-training"
df = load_results_as_df(os.path.join(HOME_DIR, "results"))
# df = df[df["select_seed_dataset"].str.endswith("200000")]
# df = df[df["select_seed_dataxset"].str.endswith("500000")]
# df = df[df["select_label_strategy"] == "auto"]
# df = df[df["select_num_pseudoexperts"] == 1]
print("Length:", len(df))


Filtered 0/322 results with no evaluation metrics
dict_keys(['data', 'full_ot_distance', 'sinkhorn_distance', 'jaccard_overlap_examples', 'jaccard_overlap_vocabulary', 'levenshtein_stats', 'args', 'expert_evaluation_metrics', 'evaluation_metrics'])
Length: 322


In [84]:
df["select_batch_fill_strategy"].value_counts()

select_batch_fill_strategy
random            90
bottomk           90
topk              89
greedy_batched    53
Name: count, dtype: int64

In [85]:
print(df["dataset"].value_counts(), "\n")
print(df["select_full_dataset_size"].value_counts(), "\n")
print(df["select_batch_fill_strategy"].value_counts(), "\n")
print(df["select_num_pseudoexperts"].value_counts(), "\n")

dataset
ag_news_10000      108
dbpedia_10000      107
newsgroup_10000    107
Name: count, dtype: int64 

select_full_dataset_size
100      69
500      64
1000     63
5000     63
10000    63
Name: count, dtype: int64 

select_batch_fill_strategy
random            90
bottomk           90
topk              89
greedy_batched    53
Name: count, dtype: int64 

select_num_pseudoexperts
1    322
Name: count, dtype: int64 



In [ ]:
df["select_projection_dim"].value_counts()

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)

dv_cols = ["dataset", "select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"]

iv_cols = ["best_eval_accuracy", "dataset_sinkhorn_distance", "dataset_jaccard_overlap_vocabulary"]

final_df = df
# final_df = df[df["select_seed_dataset"] == "nq_500000"]

final_df.groupby(dv_cols)[iv_cols].min().sort_values(by=["dataset","select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"], ascending=False)

best_eval_accuracy  \
dataset         select_seed_dataset select_full_dataset_size select_batch_fill_strategy                       
newsgroup_10000 nq_1000000          10000                    topk                                  0.157227   
                                                             random                                0.279297   
                                                             bottomk                               0.155273   
                                    5000                     topk                                  0.107422   
                                                             random                                0.245117   
                                                             bottomk                               0.110352   
                                    1000                     topk                                  0.038086   
                                                             random                                0.055664   
                                                             bottomk                               0.038086   
                                    500                      topk                                  0.036133   
                                                             random                                0.052734   
                                                             bottomk                               0.037109   
                                    100                      topk                                  0.033203   
                                                             random                                0.054688   
                                                             bottomk                               0.033203   
                nq_10000            10000                    topk                                  0.256836   
                                                             random                                0.266602   
                                                             greedy_batched                        0.264648   
                                                             bottomk                               0.259766   
                                    5000                     topk                                  0.204102   
                                                             random                                0.234375   
                                                             greedy_batched                        0.266602   
                                                             bottomk                               0.200195   
                                    1000                     topk                                  0.034180   
                                                             random                                0.060547   
                                                             greedy_batched                        0.060547   
                                                             bottomk                               0.033203   
                                    500                      topk                                  0.034180   
                                                             random                                0.061523   
                                                             greedy_batched                        0.063477   
                                                             bottomk                               0.034180   
                                    100                      topk                                  0.033203   
                                                             random                                0.060547   
                                                             greedy_batched                        0.066406   
                                                             bottomk                         